# Tutorial for learning langchain




In [ ]:
# checking up gpu
!nvidia-smi

Installation of libraries || set up env

In [ ]:
!pip install langchain
!pip install openai

from google.colab import userdata
import os
os.environ['OPENAI_API_KEY'] = userdata.get('openaikey')

# Simple import of llm and giving prompt query to it



In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.5)
name = llm("I want to open a coding club in the field of AI . suggest a creative name for it.")
print(name)

# Doing the same with prompt template which takes a input variable for the specific prompts




In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_tech = PromptTemplate(
    input_variables=['tech'],
    template="I want to open a coding club in the field of {tech} . suggest a creative name for it."
)

prompt_template_tech.format(tech="blockchain")

# For utilizing this template we hava to use chain in which we can provide the llm and template for output.& Also use run method for giving the input to the prompts

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_tech)
chain.run("DSA")

# Find every function of chain than :

In [ ]:
dir(chain)

# Use of simple sequential chain
combining of two prompt steps in which one's putput is second one's input

For ex.

input(tech)=blockchain -> CodeCrushers(generated name) -> now we want a sub team name into this club fro that i use this name as input -> geberate a sub tean names

In [ ]:
# first prompt template we already mentioned
prompt_template_tech = PromptTemplate(
    input_variables=['tech'],
    template="I want to open a coding club in the field of {tech} . suggest a creative name for it."
)

tech_chain = LLMChain(llm=llm, prompt=prompt_template_tech)

# second template
prompt_template_team =  PromptTemplate(
    input_variables=['club_name'],
    template="Suggest me a sub team names for coding club : {club_name} . return a number wise output for it."
)

team_chain = LLMChain(llm=llm, prompt=prompt_template_team)

from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [tech_chain,team_chain])

chain.run("DSA")


# Use of sequential chain
but what if i want both fist and second one's  output than :

In [ ]:
# do the same thing we done in sompleseqchain for prompt
# first prompt template we already mentioned
prompt_template_tech = PromptTemplate(
    input_variables=['tech'],
    template="I want to open a coding club in the field of {tech} . suggest a creative name for it."
)

tech_chain = LLMChain(llm=llm, prompt=prompt_template_tech, output_key='club_name')

# second template
prompt_template_team =  PromptTemplate(
    input_variables=['club_name'],
    template="Suggest me a sub team names for coding club : {club_name} . return a number wise output for it."
)

team_chain = LLMChain(llm=llm, prompt=prompt_template_team,output_key='team_name')

from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [tech_chain,team_chain],
    input_variables = ['tech'], # we have multiple input var,
    output_variables = ['club_name','team_name']
    )

chain({'tech' : 'App Development'})

# Creating Agent using llms

*   Agent is kind of mixture of one or more service that retrieves data and help the resoning part of llm to figure out answer
*   For Example :
    * We have to calculate that how many years are completed of independence day in that two service are called
    1) retrives data from wikipedia and find year of independence
    2) math function that calculate the current year - retrieved year.
    * Here it called a agent which do both of the work.


In [ ]:
!pip install wikipedia
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

tools = load_tools(["wikipedia","llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   # verbose is only for description of work how the diff steps work
    verbose=True
)
print("----------  OUTPUT STARTS FROM HERE ----------")
agent.run("How many years are completed from independence of india? ")

# Let's take a hand on serp api which is simple google search and give answer

In [ ]:
!pip install google-search-results
from google.colab import userdata

os.environ['SERPAPI_API_KEY'] = userdata.get('serpkey')

llm = OpenAI(temperature=0)

tools = load_tools(["serpapi","llm-math"],llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
    )

agent.run("first find What is GDP per capita of india and do plus 2? ")



# Memory
  # When we do the chat with chain don't have state to remember last prompts and data.
  # for that chain.memory is used you can show that by type(chain.memory)

In [ ]:
type(chain.memory)

In [ ]:
from langchain.memory import ConversationBufferMemory

# initializing it
memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template_tech, memory=memory)
name = chain.run("AI")
print(name)

In [ ]:
name = chain.run("Blockchain")
print(name)

Now show the memory

In [ ]:
chain.memory

In [ ]:
print(chain.memory.buffer)

# Conversational Chain which stoes all previous chats

In [ ]:
from langchain.chains import ConversationChain

converse = ConversationChain(llm=OpenAI(temperature=0.7))
print(converse.prompt.template)

In [ ]:
converse.run("Who won the fifa 2020?")

In [ ]:
converse.run("Who is the caption of winnig team?")


In [ ]:
converse.memory.buffer

we have only store last x memory than

In [ ]:
memory = ConversationBufferWindowMemory(k=1) #k is the value of how many we have to send or store in buffer